In [505]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [506]:
# data inporting and merging
books  = pd.read_csv('../data/BX-Books.csv', sep=";", on_bad_lines='skip', encoding='latin-1')
ratings = pd.read_csv('../data/BX-Book-Ratings.csv', sep=";", on_bad_lines='skip', encoding='latin-1')
users  = pd.read_csv("../data/BX-Users.csv", sep=";", on_bad_lines='skip', encoding='latin-1')

In [507]:
print(f"books shape: {books.shape}")
print(f"ratings shape: {ratings.shape}")
print(f"users shape: {users.shape}")

books shape: (271360, 8)
ratings shape: (1149780, 3)
users shape: (278858, 3)


In [508]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [509]:
# drop unnecessary columns and rename columns
books = books.drop(columns=['Image-URL-S','Image-URL-L'])
books = books.rename(columns={'Book-Title': 'title', 'Book-Author': 'author','ISBN': 'isbn',
                            'Year-Of-Publication': 'year', 'Publisher': 'publisher',"Image-URL-M":"img_url"})

In [510]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [511]:
# drop unnecessary columns and rename columns
ratings = ratings.rename(columns={'User-ID': 'user_id', 'Book-Rating': 'book_rating', 'ISBN': 'isbn'})

In [512]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [513]:
users = users.rename(columns={'User-ID': 'user_id', 'Location': 'location', 'Age': 'user_age'})

# 📊 Data Validation in Book Recommendation System

## Tables
- **BX-Books**: book metadata (`ISBN`, `Title`, `Author`, …)  
- **BX-Users**: user metadata (`User-ID`, `Location`, `Age`)  
- **BX-Ratings**: ratings (`User-ID`, `ISBN`, `Book-Rating`)  

## Relationships
- `BX-Ratings` links **users** and **books**.  
- One user → many books.  
- One book → many users.  
- Users and books are unique in their own tables.  

## Validation Steps
1. **ID consistency**: all `User-ID` and `ISBN` in ratings must exist in users/books.  
2. **Duplicates**: remove duplicate `(User-ID, ISBN)` pairs.  
3. **Ratings check**: decide how to handle `0` (implicit feedback).  
4. **Activity filter**: drop users/books with very few ratings.  
5. **Row counts**: verify rows before and after merges.  

## Cases
- ✅ Many users rate the same book → expected.  
- ✅ Many books rated by one user → expected.  
- ❌ Same user rating the same book multiple times.  
- ❌ Ratings referencing missing users or books.  


## Consistancy Check

In [514]:
# Keep only ratings where user exists in BX-Users
ratings = ratings[ratings["user_id"].isin(users["user_id"])]
ratings.shape
# no ratings were removed 

(1149780, 3)

In [515]:
# Keep only ratings where book exists in BX-Books
ratings = ratings[ratings["isbn"].isin(books["isbn"])]
ratings.shape
# there were 118,644 ratings removed 

(1031136, 3)

There were 118,644 rating removed due to missing book.

## Remove Duplicated

In [516]:
ratings = ratings.drop_duplicates(subset=["user_id", "isbn"])
ratings.shape


(1031136, 3)

## Tabels Merging

In [517]:
# Merge safely with user and book metadata
df = ratings.merge(users, on="user_id", how="inner").merge(books, on="isbn", how="inner")

print("Final merged dataset size:", df.shape)
print("\n \n Sample book ratings:\n", df[df["isbn"] == df["isbn"].iloc[0]][["isbn", "book_rating"]].head())


Final merged dataset size: (1031136, 10)

 
 Sample book ratings:
              isbn  book_rating
0      034545104X            0
13561  034545104X            5
23643  034545104X            0
31455  034545104X            5
35641  034545104X            9


In [518]:
df.head()

,user_id,isbn,book_rating,location,user_age,title,author,year,publisher,img_url
0,276725,034545104X,0,"tyler, texas, usa",NaN,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,"seattle, washington, usa",NaN,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,"h, new south wales, australia",16.0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,"rijeka, n/a, croatia",16.0,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,"rijeka, n/a, croatia",16.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...


## Reduce dataset size

In [519]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031136 entries, 0 to 1031135
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   user_id      1031136 non-null  int64  
 1   isbn         1031136 non-null  object 
 2   book_rating  1031136 non-null  int64  
 3   location     1031136 non-null  object 
 4   user_age     753301 non-null   float64
 5   title        1031136 non-null  object 
 6   author       1031134 non-null  object 
 7   year         1031136 non-null  object 
 8   publisher    1031134 non-null  object 
 9   img_url      1031136 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 78.7+ MB


In [520]:
# Downcast int64 → int32 and float64 → float32
df["user_id"] = pd.to_numeric(df["user_id"], downcast="integer")
df["book_rating"] = pd.to_numeric(df["book_rating"], downcast="integer")
df["user_age"] = pd.to_numeric(df["user_age"], downcast="float")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031136 entries, 0 to 1031135
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   user_id      1031136 non-null  int32  
 1   isbn         1031136 non-null  object 
 2   book_rating  1031136 non-null  int8   
 3   location     1031136 non-null  object 
 4   user_age     753301 non-null   float32
 5   title        1031136 non-null  object 
 6   author       1031134 non-null  object 
 7   year         1031136 non-null  object 
 8   publisher    1031134 non-null  object 
 9   img_url      1031136 non-null  object 
dtypes: float32(1), int32(1), int8(1), object(7)
memory usage: 63.9+ MB


# Dataset Filtering

# 📊 Data Filtering Criteria for Book Recommendation System

## Goal
The purpose of filtering is to reduce noise, handle invalid entries, and ensure that both users and books have enough interactions for meaningful recommendations. This improves data quality, model performance, and computational efficiency.

## Filtering Criteria

1. **Explicit Ratings Only**  
   - Remove ratings equal to `0` (implicit feedback).  
   - Keep only ratings greater than `0`.  

2. **Minimum Ratings per Book**  
   - Keep only books with at least **4 ratings**.  
   - Ensures books have enough feedback for collaborative filtering.  

3. **Minimum Ratings per User**  
   - Keep only users with at least **3 ratings**.  
   - Ensures each user profile is informative.  

4. **Age Validation (Optional)**  
   - Keep user ages within the range **10–100**.  
   - Removes unrealistic or missing values.  

5. **Year Validation (Optional)**  
   - Keep publication years between **1500 and the current year**.  
   - Removes invalid or placeholder values.  
## Outcome
- A cleaner, denser dataset.  
- Balanced representation of users and books.  
- Reduced sparsity and improved model training.  


In [521]:
df['book_rating'].value_counts().head(2)

book_rating
0    647294
8     91804
Name: count, dtype: int64

In [522]:
# -----------------------------
# 1. Explicit ratings only
# -----------------------------
df = df[df["book_rating"] > 0]

In [523]:
df["user_id"].value_counts()

user_id
11676     6943
98391     5691
189835    1899
153662    1845
23902     1180
          ... 
276647       1
276644       1
276637       1
276811       1
276808       1
Name: count, Length: 68091, dtype: int64

In [524]:
df['user_id'].unique().shape

(68091,)

In [525]:
# -----------------------------
# 3. Filter users with > 3 ratings
# -----------------------------

user_counts = df["user_id"].value_counts()
df = df[df["user_id"].isin(user_counts[user_counts > 3].index)]


In [526]:
df.shape

(313546, 10)

In [527]:
# -----------------------------
# 2. Filter books with >= 2 ratings
# -----------------------------

num_rating = df.groupby('title')['book_rating'].count().reset_index()
num_rating

,title,book_rating
0,A Light in the Storm: The Civil War Diary of ...,1
1,"Ask Lily (Young Women of Faith: Lily Series, ...",1
2,Dark Justice,1
3,Earth Prayers From around the World: 365 Pray...,7
4,Final Fantasy Anthology: Official Strategy Gu...,2
...,...,...
121821,Ã?Â?berleben in der WÃ?Â¼ste Danakil.,1
121822,Ã?Â?bernachten mit Stil. Die schÃ?Â¶nsten Coun...,1
121823,Ã?Â?rger mit Produkt X. Roman.,3
121824,Ã?Â?stlich der Berge.,1


In [528]:
num_rating.rename(columns= {
    "book_rating":"num_of_rating"
},inplace=True)

In [529]:
num_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,1
1,"Ask Lily (Young Women of Faith: Lily Series, ...",1
2,Dark Justice,1
3,Earth Prayers From around the World: 365 Pray...,7
4,Final Fantasy Anthology: Official Strategy Gu...,2


In [530]:
df = df.merge(num_rating , on = "title")

In [531]:
df.head()

,user_id,isbn,book_rating,location,user_age,title,author,year,publisher,img_url,num_of_rating
0,276747,0060517794,9,"iowa city, iowa, usa",25.0,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch,http://images.amazon.com/images/P/0060517794.0...,30
1,276747,0671537458,9,"iowa city, iowa, usa",25.0,Waiting to Exhale,Terry McMillan,1995,Pocket,http://images.amazon.com/images/P/0671537458.0...,23
2,276747,0679776818,8,"iowa city, iowa, usa",25.0,Birdsong: A Novel of Love and War,Sebastian Faulks,1997,Vintage Books USA,http://images.amazon.com/images/P/0679776818.0...,16
3,276747,0943066433,7,"iowa city, iowa, usa",25.0,How to Deal With Difficult People,Rick Brinkman,1995,Careertrack Inc.,http://images.amazon.com/images/P/0943066433.0...,2
4,276747,1885408226,7,"iowa city, iowa, usa",25.0,The Golden Rule of Schmoozing,Aye Jaye,1998,Listen &amp; Live Audio,http://images.amazon.com/images/P/1885408226.0...,1


In [532]:
df.shape

(313546, 11)

In [533]:
df = df[df['num_of_rating'] > 2]

In [534]:
df.head()

,user_id,isbn,book_rating,location,user_age,title,author,year,publisher,img_url,num_of_rating
0,276747,0060517794,9,"iowa city, iowa, usa",25.0,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch,http://images.amazon.com/images/P/0060517794.0...,30
1,276747,0671537458,9,"iowa city, iowa, usa",25.0,Waiting to Exhale,Terry McMillan,1995,Pocket,http://images.amazon.com/images/P/0671537458.0...,23
2,276747,0679776818,8,"iowa city, iowa, usa",25.0,Birdsong: A Novel of Love and War,Sebastian Faulks,1997,Vintage Books USA,http://images.amazon.com/images/P/0679776818.0...,16
8,276813,8478885218,6,"sitges, barcelona, spain",29.0,La Fiesta De Ralph,Lisa Jewell,0,Emece Editores,http://images.amazon.com/images/P/8478885218.0...,3
10,276822,0060096195,10,"calgary, alberta, canada",11.0,The Boy Next Door,Meggin Cabot,2002,Avon Trade,http://images.amazon.com/images/P/0060096195.0...,44


In [535]:
df['user_age'].describe()

count    142004.000000
mean         36.631954
std          13.578063
min           0.000000
25%          28.000000
50%          34.000000
75%          44.000000
max         239.000000
Name: user_age, dtype: float64

In [536]:
# -----------------------------
# 4. Age validation
# -----------------------------
df = df[(df['user_age'] >= 10) & (df['user_age'] <= 100)]


In [537]:
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df['year'].describe()

count    140948.000000
mean       1973.467116
std         211.849115
min           0.000000
25%        1993.000000
50%        1998.000000
75%        2001.000000
max        2050.000000
Name: year, dtype: float64

In [538]:
# -----------------------------
# 5. Year validation
# -----------------------------
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df = df[((df["year"] >= 1500) & (df["year"] <= 2025))]


In [539]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 139335 entries, 0 to 313509
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   user_id        139335 non-null  int32  
 1   isbn           139335 non-null  object 
 2   book_rating    139335 non-null  int8   
 3   location       139335 non-null  object 
 4   user_age       139335 non-null  float32
 5   title          139335 non-null  object 
 6   author         139335 non-null  object 
 7   year           139335 non-null  int64  
 8   publisher      139334 non-null  object 
 9   img_url        139335 non-null  object 
 10  num_of_rating  139335 non-null  int64  
dtypes: float32(1), int32(1), int64(2), int8(1), object(6)
memory usage: 10.8+ MB


In [540]:
df.shape

(139335, 11)

In [541]:
df.drop_duplicates(['user_id','title'],inplace=True)
df.shape

(138834, 11)

In [542]:
df.to_csv("cleaned_book_ratings.csv",)